In [1]:
from FileReader import FileReader

In [2]:
classInstance = FileReader
df_list = classInstance.get_dataframe() #IMPORTANT:  saves a pickle to output/simple or output/full. 

#The pickle is the same as df.



 498293668655423488

 498486826269548545

 500280249629036544

 500298588992593920

 524923293711998976

 524936793633083394

 524941720249978880

 524948206023880704

 524961721744900097

 544274544174071809

 544294893146091520

 544315472075042818

 552788945017516032

 553480082996879360

 553553288625672192

 553561170637238272

 580352273001410560

 581153923987206146

 581290271997968384

 581359544682614784

 758159624122097664

 763098277986209792

 764927075522260992

 767725956706414592

 768859780240773121

 769988636754505729

 774991078265094144

 775057555865206784

 498293668655423488

 498486826269548545

 500280249629036544

 500298588992593920

 524923293711998976

 524936793633083394

 524941720249978880

 524948206023880704

 524961721744900097

 544274544174071809

 544294893146091520

 544315472075042818

 552788945017516032

 553480082996879360

 553553288625672192

 553561170637238272

 580352273001410560

 581153923987206146

 581290271997968384

 58135954468

# IMPORTANT NOTE:

Word embeddings are expressed as pickle files.
Reading of tweet data is converted to a Pandas Dataframe format and then finally into a pickle file, located output/simple or output/full.

It is important that you first execute file_reader.py and *then* this notebook, run_tests.ipynb.

In [56]:
import sys
import os
sys.path.insert(1, "./feature-extraction/embed-extractor")
from EmbedExtractor import EmbedExtractor
sys.path.insert(1, "./feature-extraction/vulgar-extractor")
from VulgarExtractor import VulgarExtractor
sys.path.insert(1, "./feature-extraction/twitter-parser")
from TwitterParser import TwitterParser
sys.path.insert(1, "./feature-extraction/opinion-extractor/")
from OpinionExtractor import OpinionExtractor

import classifiers
import pandas as pd
import numpy as np
import json
from datetime import datetime

In [55]:
ee = EmbedExtractor()

KeyboardInterrupt: 

In [41]:
def create_opinion_column(df):
    #add a binary column where opinion == 1 if the tweet text contains a strongly subjective word
    global strongly_subj_list
    OpinionExtractor.add_opinion_column(df, strongly_subj_list)

def extract_user_column(row):
    global user_labels
        
    user_dict = row['user']
    
    for col in user_dict.keys():
        user_labels.append(col)

def update_user_column(row):
    global user_vals
    user_dict = row['user']
    
    for key in user_vals.keys():
#         print(key)
        concat_key = key[5:]
        if concat_key in user_dict:
#             print(sup)
            val = user_dict[concat_key]
            user_vals[key].append(val)
        else:
            user_vals[key].append(np.nan)
            
def convert_date(row):
    date = row['user_created_at'].split()
    date_str = ' '.join([date[1], date[2], date[-1]])
    
    datetime_object = datetime.strptime(date_str, '%b %d %Y')
    date_int = datetime_object.year * 10000 + datetime_object.month * 100 + datetime_object.day
        
    return date_int

def convert_to_int(row, col):
    return int(row[col])

def normalize_column(df, col):
    col_array = np.asarray(df[col].tolist())
    mean = np.mean(col_array)
    std = np.std(col_array)
    col_array = (col_array - mean) / float(std)
    
    df[col] = col_array
    
def create_user_features(df):

    global user_labels
    global user_vals
    user_labels = []
    df.apply(extract_user_column, axis = 1)
    user_labels = ['user_' + label for label in set(user_labels)]
    user_vals = {label:[] for label in user_labels}
    
    df.apply(update_user_column, axis = 1) 
    
    user_df = pd.DataFrame(user_vals)
    user_df['user_created'] = user_df.apply(convert_date, axis = 1)

    col_list = ['user_default_profile', 
                'user_favourites_count', 'user_followers_count', 'user_friends_count', 'user_geo_enabled',
                'user_listed_count', 'user_statuses_count', 'user_verified','user_created']

    for col in col_list:
        user_df[col] = user_df.apply(lambda x : convert_to_int(x, col), axis = 1)

    # normalize_column(user_df, col)
    user_df = user_df[col_list]    

    norm_list = ['user_favourites_count', 'user_followers_count', 'user_friends_count',
                'user_listed_count', 'user_statuses_count','user_created']

    for col in norm_list:
        normalize_column(user_df, col)

    df = pd.concat([df.reset_index(), user_df], axis = 1)

    return df    

In [50]:
def normalize(column_name, df):
    std = df[column_name].std()
    norm_col = df[column_name].apply(lambda x: x - std)
    df[column_name] = norm_col

# builds the labels and vectorizations of given data
#if you want to fool around with including/excluding certain features and whatnot, this is the place to do it

def labels_and_vectors(file, index=0):
    df = pd.read_pickle(file)
    
    wordlist = VulgarExtractor.vulgarWords("feature-extraction/vulgar-extractor/badwords.txt") 
    dftext = df[['text']]
    result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
    df['isVulgar'] = result

    word_embeddings = [ee.tweetVec(tagged_line) for tagged_line in df['text']]
    textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
    tagged_sents = TwitterParser.tag(textlist)
    df['POS'] = tagged_sents

    processed_sents = []
    for tagged_sent in df['POS']:
        processed_words = []
        for word, tag in tagged_sent:
            if tag == 'U':
                processed_words.append('someurl')
            elif tag == '@':
                processed_words.append('@someuser')
            else:
                processed_words.append(word)
        sent = ' '.join(processed_words)
        processed_sents.append(sent)
    df['text'] = processed_sents

    word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in df['POS']]
    pos_count_list = [TwitterParser.pos_counts(tagged_line) for tagged_line in df['POS']]
    contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in df['POS']]
    contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in df['POS']]
    contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in df['POS']]
    contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in df['POS']]

    df['wordCount'] = word_counts
    df['posCounts'] = pos_count_list
    df['containsAdjective'] = contains_adjs
    df['containsURL'] = contains_urls
    df['containsEmoji'] = contains_emojis
    df['containsAbbreviation'] = contains_abbrevs
    df['wordEmbedding'] = word_embeddings

    
    for i, tag in enumerate(TwitterParser.tagset):
        tag_counts = []
        for pos_counts in df['posCounts']:
            tag_counts.append(pos_counts[i])
        column_name = 'num_' + tag
        df[column_name] = tag_counts
        normalize(column_name, df)
    
    global strongly_subj_list
    strongly_subj_list = OpinionExtractor.initialize_subjectivity()
    create_opinion_column(df)
    df = create_user_features(df)    
        
    # Changes "true"/"false"/"unverified" to numeric values, just like the in the early cells
    df.loc[df.classification == 'true', 'classification'] = 1
    df.loc[df.classification == 'false', 'classification'] = 0
    df.loc[df.classification == 'unverified', 'classification'] = 2
    # getting the labels

    #removed containsURL
    attributes = ['isVulgar', 'containsAdjective', 'containsURL', 'containsEmoji', 'containsAbbreviation', 'wordCount']
    for tag in TwitterParser.tagset:
        attributes.append('num_' + tag)

        
    attributes = attributes + ['num_replies', 're_has_?', 're_has_NOT', 're_has_correct',
 're_has_credib', 're_has_data', 're_has_detail', 're_has_fabricat', 're_has_lie', 're_has_proof', 
                  're_has_source', 're_has_witness', 'opinion', 'user_default_profile',
 'user_favourites_count', 'user_followers_count', 'user_friends_count', 'user_geo_enabled', 'user_listed_count', 
                  'user_statuses_count', 'user_verified', 'user_created']
#     print(df.columns.values)
        
        
    labels = df['classification']
    labels = [l for l in labels]
    labels = np.array(labels)


    # getting the values as a list of lists
    values = df[attributes].values.tolist()
    word_embedding_values = df['wordEmbedding'].values.tolist()


#     #Below puts the tweet ID as a feature. Comment this out if you aren't using tweetID
#     for i,index in enumerate(df.index):
#        dev_values[i].append(int(index))


#UNCOMMENT THIS IN ORDER TO INCOPORATE WORD_EMBEDDINGS AGAIN
    for i,d in enumerate(word_embedding_values):
        values[i].extend(d)

    values = np.array(values)
    if index == 1:
        return df.index, values
    

    
    return labels, values
    

In [51]:
# note that the indices of labels-to-values should not be mismatched
tr_labels, tr_values = labels_and_vectors('output/full/train_data_full.pickle')
indices, dev_values = labels_and_vectors('output/full/goldtest_data_full.pickle', index=1)

In [52]:
tr_values.shape

(272, 9053)

In [28]:
for i in tr_values[0]:
    print(i)

0.0
0.0
0.0
0.0
0.0
15.0
1.21090067806
0.395958639797
-0.20718062552
0.344229689792
-0.257274244534
-0.213723590074
0.0
1.28591617725
-0.955992546787
-0.725216373781
-0.158636405053
1.87047343621
1.61904249549
-0.421615213065
-0.289433891287
-0.158636405053
0.0
0.261079595325
-0.374298417583
-0.224285276806
-0.762058477748
0.0
-0.663311868671
0.567789682129
-0.134577214633
-0.797020018101
-0.0471069999039
-0.162180006504
-0.0801519975066
-0.395770013332
0.657090008259
0.245309993625
-0.793649971485
0.189419999719
-0.0713059976697
0.634869992733
-0.203050002456
-0.129280000925
0.0991029962897
0.615849971771
-0.134489998221
-0.301739990711
0.188690006733
-0.437409996986
0.526970028877
0.152050003409
1.16760003567
0.399269998074
0.245600000024
0.393629997969
-0.0628380030394
0.188280001283
-0.371919989586
-0.0114609999582
-1.50660002232
0.0298950001597
0.26506999135
-0.441379994154
-0.526700019836
0.413659989834
-0.926240026951
0.0577379986644
0.0535369999707
-0.299549996853
0.17502999305

1.67990005016
0.130930006504
-0.0590730011463
-0.608420014381
0.448339998722
0.0359750017524
-0.142680004239
0.224820002913
-0.0924670025706
-0.168190002441
0.100189998746
0.0455759987235
-0.319350004196
0.491910010576
-0.0115269999951
-0.55505001545
0.113889999688
0.0900619998574
0.114849999547
0.313109993935
-0.381749987602
0.224940001965
0.131679996848
0.300680011511
-0.655990004539
0.446530014277
0.445710003376
0.226520001888
0.63058000803
-0.591170012951
0.323179990053
-0.134739995003
-0.0592110008001
-0.170870006084
0.182170003653
0.296609997749
1.23049998283
0.345070004463
-0.217690005898
-0.0706669986248
-0.775839984417
-0.459879994392
-0.310779988766
0.134320005774
0.239610001445
0.214880004525
0.126739993691
-0.123429998755
-0.43656000495
-0.0570270009339
0.397430002689
0.361759990454
-0.392219990492
-0.265089988708
-0.141190007329
0.0138569995761
0.00132100004703
-0.38870999217
0.134090006351
0.0277679990977
-0.517220020294
-0.0392839983106
-0.0955530032516
-0.225639998913
0

-0.0293829999864
0.283769994974
0.303050011396
0.157900005579
-0.50707000494
0.396160006523
-0.21425999701
-0.363599985838
0.190579995513
0.396600008011
0.907140016556
0.421149998903
-0.0493730008602
-0.331690013409
-0.459179997444
0.404269993305
0.393880009651
0.102559998631
-0.18436999619
-0.235100001097
0.156589999795
-0.183379992843
0.394650012255
-0.254509985447
0.257699996233
-0.209089994431
-0.180549994111
-0.198559999466
-0.231199994683
0.634060025215
-0.034795999527
0.476040005684
-0.507629990578
0.0624370016158
-0.337760001421
0.370240002871
-0.268620014191
-0.662829995155
0.503600001335
0.68044000864
0.448740005493
0.189070001245
-0.0331370010972
-0.227880001068
-0.391019999981
0.351049989462
1.49440002441
-0.440050005913
-0.843550026417
0.160530000925
-0.255259990692
-0.0873849987984
-0.00309040001594
0.0589610002935
-0.501030027866
0.169499993324
0.289970010519
-0.0648989975452
-0.214579999447
-0.171210005879
0.359450012445
0.586090028286
-0.480260014534
0.136309996247
0.0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [53]:
#change classifier here
predictions, probabilities = classifiers.naive_bayes(tr_values, tr_labels, dev_values)
ps = []

# need to convert the numerical predictions back into their string values
for i, p in enumerate(predictions):
    if p == 0:
        ps.append('false')
    if p == 1:
        ps.append('true')
    if p == 2:
        ps.append('unverified')

# creates pairings of the prediction and the probability of the prediction
pred_probs_pairs = [[ps[i], probabilities[i][predictions[i]]] for i in range(len(predictions))] 
#attaches the tweetID (called reference_id in the score.py file)
pred_dict = {index:pred_probs_pairs[i] for i,index in enumerate(indices)}

In [57]:
output_dir = './output/classifier_output/'
try:
    os.stat(output_dir)
except:
    os.mkdir(output_dir)  

with open('output/classifier_output/goldtest_nb.json', 'w') as outfile:
    json.dump(pred_dict, outfile)

In [58]:
!python3 scorer/score.py data/semeval2017-task8-dataset/goldtest/subtaskb.json output/classifier_output/goldtest_nb.json

28 entries in reference file
unmatched entry: 775057555865206784 -- no reference value for this document
unmatched entry: 498486826269548545 -- no reference value for this document
unmatched entry: 774991078265094144 -- no reference value for this document
unmatched entry: 524948206023880704 -- no reference value for this document
unmatched entry: 769988636754505729 -- no reference value for this document
unmatched entry: 544315472075042818 -- no reference value for this document
unmatched entry: 524936793633083394 -- no reference value for this document
unmatched entry: 553561170637238272 -- no reference value for this document
unmatched entry: 763098277986209792 -- no reference value for this document
unmatched entry: 498293668655423488 -- no reference value for this document
unmatched entry: 524941720249978880 -- no reference value for this document
unmatched entry: 553553288625672192 -- no reference value for this document
unmatched entry: 764927075522260992 -- no reference value f